In [ ]:
# GUOQ

In [1]:
import os
import subprocess
from pathlib import Path
from concurrent.futures import ProcessPoolExecutor, as_completed
import multiprocessing
import psutil
import signal
import time

In [2]:

# 1) Clamp any BLAS/OMP threading (if your scripts import numpy, etc.)
os.environ["OPENBLAS_NUM_THREADS"] = "1"
os.environ["OMP_NUM_THREADS"]      = "1"

# 3) Worker function
def run_one(circ_path: str, output_dir: str = "../results", timeout: int = 120, gate: str = "NAM"):
    circ = Path(circ_path)
    cmd = [
        # "timeout", str(timeout),
        "java",
        "-cp", "target/GUOQ-1.0-jar-with-dependencies.jar",
        "qoptimizer.Optimizer",
        "-g", gate,
        "-opt", "T",
        "-resynth", "BQSKIT",
        "-eps", "1e-10",
        "--verbosity", "3",
        "-out", output_dir,
        circ_path,
    ]
    log_file = Path(output_dir) / f"{circ.stem}_logs.txt"
    log_file.parent.mkdir(parents=True, exist_ok=True)
    with open(log_file, "w") as f:
        try:
            subprocess.run(cmd, stdout=f, stderr=subprocess.STDOUT, check=True, timeout=120)
            return circ.name, "OK"
        except subprocess.TimeoutExpired:
            return circ.name, "TIMEOUT"
        except subprocess.CalledProcessError as e:
            return circ.name, f"ERROR({e.returncode})"

In [3]:
from typing import List

workers = 5


def call_parallel(gate:str = "NAM", output_dir: str = "../results/", circs: List = []):

    print(f"Launching up to {workers} parallel optimizer jobs...")
    results = []
    with ProcessPoolExecutor(max_workers=workers) as pool:
        futures = {pool.submit(run_one, c, output_dir, gate): c for c in circs}
        for fut in as_completed(futures):
            name, status = fut.result()
            print(f"{name}: {status}")
            results.append((name, status))

In [4]:
def start_resynth():
    return subprocess.Popen(
        ["python", "resynth.py", "--bqskit", "--bqskit_auto_workers"],
        preexec_fn=os.setsid, 
    )

def interrupt_and_restart(proc):
    os.killpg(proc.pid, signal.SIGINT)
    time.sleep(5)
    if proc.poll() is None:
        os.killpg(proc.pid, signal.SIGKILL)
    return start_resynth()

In [5]:
root = Path("benchmarks/ibmnew")
wd = Path("")
results_dir = Path("../results/ibmnew")

done = {
    p.name.replace("latest_sol__", "")
    for p in results_dir.glob("latest_sol__*.qasm")
}

circs = []
for p in root.rglob("*.qasm"):
    if p.is_file() and p.name not in done:
        circs.append(str(p.relative_to(wd)))
        
print(len(circs))
# chunk_size = 25
# proc = start_resynth()
# time.sleep(10)

# for i in range(0, len(circs), chunk_size):
#     call_parallel("IBMN", "../results/ibmnew", circs[i:i + chunk_size])
#     proc = interrupt_and_restart(proc)
#     while psutil.virtual_memory().used / 1e9 > 16:
#         time.sleep(5)

# os.killpg(proc.pid, signal.SIGINT)
# time.sleep(5)

0


In [6]:
import psutil, time

root = Path("benchmarks/ion")
wd = Path("")
results_dir = Path("../results/ion")

done = {
    p.name.replace("latest_sol__", "")
    for p in results_dir.glob("latest_sol__*.qasm")
}

circs = []
for p in root.rglob("*.qasm"):
    if p.is_file() and p.name not in done:
        circs.append(str(p.relative_to(wd)))

print(len(circs))

# chunk_size = 25
# proc = start_resynth()
# time.sleep(10)

# for i in range(0, len(circs), chunk_size):
#     call_parallel("ION", "../results/ion", circs[i:i + chunk_size])
#     proc = interrupt_and_restart(proc)
#     while psutil.virtual_memory().used / 1e9 > 16:
#         time.sleep(5)

# os.killpg(proc.pid, signal.SIGINT)
# time.sleep(5)

0


In [7]:
import psutil, time

root = Path("benchmarks/ibm")
wd = Path("")
results_dir = Path("../results/ibm")

done = {
    p.name.replace("latest_sol__", "")
    for p in results_dir.glob("latest_sol__*.qasm")
}

circs = []
for p in root.rglob("*.qasm"):
    if p.is_file() and p.name not in done:
        circs.append(str(p.relative_to(wd)))

print(len(circs))
# chunk_size = 25
# proc = start_resynth()
# time.sleep(10)

# for i in range(0, len(circs), chunk_size):
#     call_parallel("IBMO", "../results/ibm", circs[i:i + chunk_size])
#     proc = interrupt_and_restart(proc)
#     while psutil.virtual_memory().used / 1e9 > 16:
#         time.sleep(5)

# os.killpg(proc.pid, signal.SIGINT)
# time.sleep(5)

0


In [5]:
root = Path("benchmarks/nam_t_tdg")
wd = Path("")
results_dir = Path("../results/nam_t")

done = {
    p.name.replace("latest_sol__", "")
    for p in results_dir.glob("latest_sol__*.qasm")
}

circs = []
for p in root.rglob("*.qasm"):
    if p.is_file() and p.name not in done:
        circs.append(str(p.relative_to(wd)))

print(len(circs))
        
chunk_size = 25
proc = start_resynth()
time.sleep(10)

for i in range(0, len(circs), chunk_size):
    call_parallel("NAM", "../results/guoq_results/nam_t", circs[i:i + chunk_size])
    proc = interrupt_and_restart(proc)
    while psutil.virtual_memory().used / 1e9 > 16:
        time.sleep(5)

os.kill(proc.pid, signal.SIGINT)
time.sleep(5)

251
Launching up to 5 parallel optimizer jobs...


127.0.0.1 - - [07/May/2025 19:49:15] "POST /bqskit HTTP/1.1" 200 -
127.0.0.1 - - [07/May/2025 19:49:36] "POST /bqskit HTTP/1.1" 200 -


{'circuit': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[3];\ncx q[1],q[2];\nt q[0];\nt q[2];\nh q[1];\ncx q[2],q[0];\nt q[1];\ncx q[1],q[2];\ntdg q[2];\ncx q[0],q[1];\ncx q[0],q[2];\nt q[1];\ntdg q[2];\ntdg q[0];\ncx q[1],q[2];\ncx q[0],q[1];\ncx q[2],q[0];\nh q[1];\ncx q[0],q[2];\nt q[1];\nt q[2];\n', 'original_size': 20, 'original_2q_size': 9, 'bqskit_params': {'opt_level': 3, 'epsilon': 1e-12}, 'resynth_size': 17, 'resynth_2q_size': 6, 'resynthesized_circuit': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[3];\nrz(1.9823708415384145) q[0];\nrz(-pi/2) q[0];\nh q[0];\nrz(-pi/2) q[0];\nrz(pi) q[0];\nrz(-pi/2) q[0];\nh q[0];\nrz(-pi/2) q[0];\nrz(14.510990589808854) q[0];\nrz(4.712388967123891) q[1];\nrz(-pi/2) q[1];\nh q[1];\nrz(-pi/2) q[1];\nrz(4.712389019933655) q[1];\nrz(-pi/2) q[1];\nh q[1];\nrz(-pi/2) q[1];\nrz(13.837757531707114) q[1];\nrz(4.174783620445887) q[2];\nrz(-pi/2) q[2];\nh q[2];\nrz(-pi/2) q[2];\nrz(4.712388967232048) q[2];\nrz(-pi/2) q[2];\nh q[2];\nrz(-pi/2) q[2];\nr

127.0.0.1 - - [07/May/2025 19:49:41] "POST /bqskit HTTP/1.1" 200 -


{'circuit': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[3];\nt q[2];\nt q[0];\nt q[1];\nt q[2];\nt q[0];\nt q[1];\ntdg q[2];\ntdg q[0];\ntdg q[1];\ntdg q[2];\ntdg q[0];\ntdg q[1];\nh q[2];\nh q[0];\nh q[1];\ntdg q[2];\ntdg q[0];\ntdg q[1];\ntdg q[2];\ntdg q[0];\ntdg q[1];\nt q[2];\nt q[0];\nt q[1];\nt q[2];\nt q[0];\nt q[1];\ntdg q[2];\ntdg q[0];\ntdg q[1];\ntdg q[2];\ntdg q[0];\ntdg q[1];\nh q[2];\nh q[0];\nh q[1];\ntdg q[2];\ntdg q[0];\ntdg q[1];\ntdg q[2];\ntdg q[0];\ntdg q[1];\nt q[2];\nt q[0];\nt q[1];\nt q[2];\nt q[0];\nt q[1];\nt q[2];\ntdg q[0];\nt q[1];\nh q[2];\ntdg q[0];\nt q[1];\nt q[2];\ntdg q[0];\nt q[1];\nh q[2];\ntdg q[0];\nh q[1];\nt q[2];\nt q[0];\ntdg q[1];\nh q[2];\nt q[0];\nh q[1];\ntdg q[2];\ntdg q[0];\ntdg q[1];\nh q[2];\ntdg q[0];\nh q[1];\ntdg q[2];\nh q[0];\nt q[1];\nh q[2];\ntdg q[0];\nh q[1];\ntdg q[2];\ntdg q[0];\nt q[1];\nh q[2];\ntdg q[0];\nh q[1];\ntdg q[2];\ntdg q[0];\nt q[1];\nh q[2];\ntdg q[0];\nh q[1];\nt q[2];\ntdg q[0];\nt q[1];\nh q[2];\nh q[0];

...

{'circuit': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[3];\nh q[1];\nh q[0];\nt q[2];\nt q[1];\nh q[0];\ncx q[1],q[2];\nt q[0];\ncx q[0],q[1];\ntdg q[1];\ncx q[2],q[0];\ncx q[2],q[1];\nt q[0];\ntdg q[1];\ntdg q[2];\ncx q[0],q[1];\ncx q[2],q[0];\ncx q[1],q[2];\nh q[0];\nh q[1];\nt q[1];\n', 'original_size': 20, 'original_2q_size': 7, 'bqskit_params': {'opt_level': 3, 'epsilon': 1e-12}, 'resynth_size': 21, 'resynth_2q_size': 6, 'resynthesized_circuit': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[3];\nrz(3.00259559829521) q[0];\nrz(-pi/2) q[0];\nh q[0];\nrz(-pi/2) q[0];\nrz(7.853981633941311) q[0];\nrz(-pi/2) q[0];\nh q[0];\nrz(-pi/2) q[0];\nrz(10.995574287486946) q[0];\nrz(1.0612989657803877) q[1];\nrz(-pi/2) q[1];\nh q[1];\nrz(-pi/2) q[1];\nrz(6.283185306058625) q[1];\nrz(-pi/2) q[1];\nh q[1];\nrz(-pi/2) q[1];\nrz(10.486076926550718) q[1];\nrz(5.870380757839489) q[2];\nrz(-pi/2) q[2];\nh q[2];\nrz(-pi/2) q[2];\nrz(7.85398163396724) q[2];\nrz(-pi/2) q[2];\nh q[2];\nrz(-pi/2) q[2];\n

127.0.0.1 - - [07/May/2025 21:31:23] "POST /bqskit HTTP/1.1" 200 -


4gt4-v0_72-checkpoint.qasm: TIMEOUT


Compiler interrupted.


{'circuit': 'OPENQASM 2.0;\ninclude "qelib1.inc";\nqreg q[3];\nrz(1.843239688696968) q[2];\nrz(3.1415926434269608) q[1];\nrz(3.4972161929127212) q[0];\nrz(-1.5707963267948966) q[2];\nrz(-1.5707963267948966) q[1];\nrz(-1.5707963267948966) q[0];\nh q[2];\nh q[1];\nh q[0];\nrz(-1.5707963267948966) q[2];\nrz(-1.5707963267948966) q[1];\nrz(-1.5707963267948966) q[0];\nrz(9.42477796076538) q[2];\nrz(7.85398161320115) q[1];\nrz(7.853981627558054) q[0];\nrz(-1.5707963267948966) q[2];\nrz(-1.5707963267948966) q[1];\nrz(-1.5707963267948966) q[0];\nh q[2];\nh q[1];\nh q[0];\nrz(-1.5707963267948966) q[2];\nrz(-1.5707963267948966) q[1];\nrz(-1.5707963267948966) q[0];\nrz(14.65012173009907) q[2];\nrz(11.764562010981413) q[1];\nrz(12.56637057592593) q[0];\ncx q[1],q[0];\nrz(0.8018086043890665) q[1];\nrz(3.141592608187778) q[0];\nrz(-1.5707963267948966) q[1];\nrz(-1.5707963267948966) q[0];\nh q[1];\nh q[0];\nrz(-1.5707963267948966) q[1];\nrz(-1.5707963267948966) q[0];\nrz(7.853981614946139) q[1];\nrz(4

In [9]:
import psutil, time

root = Path("benchmarks/nam_rz")
wd = Path("")
results_dir = Path("../results/nam_rz")

done = {
    p.name.replace("latest_sol__", "")
    for p in results_dir.glob("latest_sol__*.qasm")
}

circs = []
for p in root.rglob("*.qasm"):
    if p.is_file() and p.name not in done:
        circs.append(str(p.relative_to(wd)))

print(len(circs))

# chunk_size = 25
# proc = start_resynth()
# time.sleep(10)

# for i in range(0, len(circs), chunk_size):
#     call_parallel("NAM", "../results/nam_rz", circs[i:i + chunk_size])
#     proc = interrupt_and_restart(proc)
#     while psutil.virtual_memory().used / 1e9 > 16:
#         time.sleep(5)

# os.killpg(proc.pid, signal.SIGINT)
# time.sleep(5)

0


In [ ]:
# Tket